In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Is this notebook running on Colab or Kaggle?
IS_COLAB = "google.colab" in sys.modules
IS_KAGGLE = "kaggle_secrets" in sys.modules

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

# TensorFlow ≥2.0 is required
import tensorflow as tf
from tensorflow import keras
assert tf.__version__ >= "2.0"

if not tf.config.list_physical_devices('GPU'):
    print("No GPU was detected. LSTMs and CNNs can be very slow without a GPU.")
    if IS_COLAB:
        print("Go to Runtime > Change runtime and select a GPU hardware accelerator.")
    if IS_KAGGLE:
        print("Go to Settings > Accelerator and select GPU.")

# Common imports
import numpy as np
import pandas as pd
import sklearn.preprocessing
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import os
import datetime as dt
from pathlib import Path

# to make this notebook's output stable across runs
np.random.seed(42)
tf.random.set_seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

In [27]:
arti_train=pd.read_csv('../input/h-and-m-personalized-fashion-recommendations/articles.csv', dtype={"article_id": "str"})
cus_train=pd.read_csv('../input/h-and-m-personalized-fashion-recommendations/customers.csv')

In [28]:
cus_train

In [33]:
trans_train=pd.read_csv('../input/h-and-m-personalized-fashion-recommendations/transactions_train.csv')
trans_train.head()

In [34]:
customer = trans_train.customer_id.unique()
customer_map = {i:val for i,val in enumerate(customer)}
inverse_customer_map = {val:i for i,val in enumerate(customer)}
trans_train['customer_id']=trans_train['customer_id'].map(inverse_customer_map)
trans_train

In [35]:
trans_train['article_id'] = trans_train.article_id.astype('int32')
trans_train.t_dat = pd.to_datetime(trans_train.t_dat)
trans_train = trans_train[['t_dat','customer_id','article_id']]
trans_train_original = trans_train
print( trans_train.shape )
trans_train.tail()

In [36]:
cus_train = cus_train[['customer_id','age']]
customer = cus_train.customer_id.unique()
customer_map = {i:val for i,val in enumerate(customer)}
inverse_customer_map = {val:i for i,val in enumerate(customer)}
cus_train['customer_id']=cus_train['customer_id'].map(inverse_customer_map)
cus_train.tail()

In [37]:
arti_train=arti_train[['article_id','product_type_no','graphical_appearance_no','colour_group_code']]
arti_train['article_id'] = arti_train.article_id.astype('int32')
arti_train.head()

In [38]:
pucr = trans_train.groupby('customer_id').t_dat.max().reset_index()
pucr.columns = ['customer_id','max_dat']

In [39]:
pucr

In [40]:
trans_train = trans_train.merge(pucr,on=['customer_id'],how='left')
trans_train['diff_dat'] = (trans_train.max_dat - trans_train.t_dat).dt.days
trans_train = trans_train.loc[trans_train['diff_dat']<=14]

In [41]:
trans_train['diff_dat'].unique()

In [42]:
pucr = trans_train.groupby(['customer_id','article_id'])['t_dat'].agg('count').reset_index() 
pucr.columns = ['customer_id','article_id','count']

In [43]:
pucr

In [44]:
trans_train = trans_train.merge(pucr,on=['customer_id','article_id'],how='left')
trans_train = trans_train.sort_values(['count','t_dat'],ascending=False)
trans_train

In [45]:
trans_train = trans_train.drop_duplicates(['customer_id','article_id'])
trans_train = trans_train.sort_values(['count','t_dat'],ascending=False)
trans_train= trans_train.reset_index(drop=True)
trans_train

In [46]:
trans_train=trans_train.reset_index(drop=False)
trans_train

In [47]:
cust_age=pd.merge(trans_train, cus_train, on='customer_id')
cust_age.head()

In [49]:
cust_age=cust_age[['index','customer_id','age','article_id']]
cust_age=cust_age.fillna({'age':18})
cust_age

In [51]:
art_sel = pd.merge(cust_age, arti_train, on='article_id')
art_sel

In [ ]:
#Final X  

In [52]:
output1=art_sel[['age','product_type_no','graphical_appearance_no','colour_group_code']]
output1.head()

In [ ]:
#final Y

In [53]:
output2=art_sel[['article_id']]
output2.head()

In [ ]:
#neural network preprocessing 

In [55]:
#Import required libraries 
import keras #library for neural network
import pandas as pd #loading data in table form  
import seaborn as sns #visualisation 
import matplotlib.pyplot as plt #visualisation
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.preprocessing import normalize

In [56]:
X_normalized=normalize(output1,axis=0)
print("Examples of X_normalised\n",X_normalized[:3])

In [58]:
#Creating train,test and validation data
'''
80% -- train data
20% -- test data
'''
total_length=len(output1)
train_length=int(0.8*total_length)
test_length=int(0.2*total_length)

X_train=X_normalized[:train_length]
X_test=X_normalized[train_length:]
y_train=output2[:train_length]
y_test=output2[train_length:]

print("Length of train set x:",X_train.shape[0],"y:",y_train.shape[0])
print("Length of test set x:",X_test.shape[0],"y:",y_test.shape[0])

In [61]:
#Neural network module
from keras.models import Sequential 
from keras.layers import Dense,Activation,Dropout 
#from keras.layers.normalization import Batch_Normalization 
from keras.utils import np_utils

In [65]:
model=Sequential()
model.add(Dense(1000,input_dim=4,activation='relu'))
model.add(Dense(500,activation='relu'))
model.add(Dense(300,activation='relu'))
model.add(Dropout(0.2))
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [66]:
model.summary()

In [67]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),batch_size=20,epochs=2,verbose=1)

In [ ]:
transactions['price'].describe()

In [ ]:
transac = transactions['2018-10-01':'2019-01-01']
transac

In [ ]:
transac

In [ ]:
transac1=pd.read_csv('../input/h-and-m-personalized-fashion-recommendations/transac.csv')
transac1.head()

In [ ]:
customer = transactions.customer_id.unique()

In [ ]:
customer_map = {i:val for i,val in enumerate(customer)}

In [ ]:
inverse_customer_map = {val:i for i,val in enumerate(customer)}

In [ ]:
article = transactions.article_id.unique()
article_map = {i:val for i,val in enumerate(article)}
inverse_article_map = {val:i for i,val in enumerate(article)}

In [ ]:
transactions['customer_id']=transactions['customer_id'].map(inverse_customer_map)
transactions['old_article_id']=transactions['article_id']

In [ ]:
transactions['article_id']=transactions['article_id'].map(inverse_article_map)

In [ ]:
train=transactions['2018-01-01':'2019-01-01']
test=transactions['2018-01-01':'2019-01-01']
train


In [ ]:
user_id_input = keras.layers.Input(shape=[1], name='user')
item_id_input = keras.layers.Input(shape=[1], name='item')
embedding_size = 10 # 5

In [ ]:
user_embedding = keras.layers.Embedding(output_dim=embedding_size, input_dim=customer.shape[0],
                           input_length=1, name='user_embedding')(user_id_input)
item_embedding = keras.layers.Embedding(output_dim=embedding_size, input_dim=article.shape[0],
                           input_length=1, name='item_embedding')(item_id_input)

In [ ]:
user_vecs = keras.layers.Reshape([embedding_size])(user_embedding)
item_vecs = keras.layers.Reshape([embedding_size])(item_embedding)
input_vecs = keras.layers.Concatenate()([user_vecs, item_vecs])

In [ ]:
x = keras.layers.Dense(128, activation='relu')(input_vecs)
y = keras.layers.Dense(1)(x)

In [ ]:
model = keras.models.Model(inputs=[user_id_input, item_id_input], outputs=y)
model.compile(optimizer='adam', loss='mse')
history = model.fit([train["customer_id"], train["article_id"]]
                    , train["price"]
                    , batch_size=64, epochs=10, validation_split=0.1)


In [ ]:
submission = pd.read_csv('../input/h-and-m-personalized-fashion-recommendations/sample_submission.csv')

In [ ]:
# submission = submission['customer_id']

In [ ]:
test = submission[:10000]

In [ ]:
test

In [ ]:
y_pred = model.predict(test)